In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
%matplotlib inline
from sklearn import metrics
from xgboost import XGBRegressor

In [3]:
df = pd.read_excel('training_data.xlsx')
dft = pd.read_excel('test_data.xlsx')


In [4]:
cols = df.isnull().mean()*100
main = []
for i in range(len(cols)):
    if cols[i] < 40:
        main.append(cols.index[i])

In [5]:
df_final = df[main]

In [6]:
X = df_final.drop(['Mortality','ID'], axis = 1)

In [7]:
y = df_final['Mortality']

In [8]:
X.fillna(X.mean(), inplace=True)
X['Severity'].fillna('All', inplace=True)
X['Ventilation'].fillna('Both', inplace=True)

In [9]:
X.head()

,Severity,Ventilation,% Male,Hypertension,Diabetes,Cardiovascular Disease (incl. CAD),Fever (temperature ≥37·3°C),Cough,Lymphocyte Count (10^9/L) - Median
0,All,Both,0.6230,0.300000,0.1900,0.08000,0.9400,0.790000,1.0
1,Severe/Critical Only,Both,0.7037,0.481500,0.3148,0.24070,0.9444,0.722200,0.6
2,All,Both,0.5912,0.233600,0.1387,0.01460,0.9416,0.817500,1.1
3,All,Both,0.6030,0.286662,0.1160,0.14218,0.9150,0.603716,0.9
4,All,Both,0.6160,0.286662,0.1010,0.14218,0.8990,0.603716,0.8


In [11]:
df['Severity'].value_counts()

All                     211
Severe/Critical Only    144
Mild only                91
Both                      5
Mild                      2
Severe                    2
Severe/critical only      2
Asymptomatic only         1
Name: Severity, dtype: int64

In [14]:
df['Severity'] = df['Severity'].replace('Severe/Critical Only','Severe')
df['Severity'] = df['Severity'].replace('Severe/critical only','Severe')
df['Severity'] = df['Severity'].replace('Mild only','Mild')

In [15]:
df['Severity'].value_counts()

All                  211
Severe               148
Mild                  93
Both                   5
Asymptomatic only      1
Name: Severity, dtype: int64

In [16]:
test_df = dft[X.columns]

In [21]:
test_df.fillna(test_df.mean(), inplace=True)
test_df['Severity'].fillna('All', inplace=True)
test_df['Ventilation'].fillna('Both', inplace=True)

In [22]:
test_df['Severity'].value_counts()

All                     36
Severe/Critical Only     9
Both                     4
Mild only                2
Severe/critical only     1
Mild Only                1
Name: Severity, dtype: int64

In [24]:
test_df['Severity'] = test_df['Severity'].replace('Severe/Critical Only','Severe')
test_df['Severity'] = test_df['Severity'].replace('Severe/critical only','Severe')
test_df['Severity'] = test_df['Severity'].replace('Mild Only','Mild')
test_df['Severity'] = test_df['Severity'].replace('Mild only','Mild')

In [26]:
test_df['Severity'].value_counts()

All       36
Severe    10
Both       4
Mild       3
Name: Severity, dtype: int64

In [28]:
from sklearn.preprocessing import LabelEncoder
lr = LabelEncoder()
X['Ventilation'] = lr.fit_transform(X['Ventilation'])
test_df['Ventilation'] = lr.transform(test_df['Ventilation'])



In [29]:
X['Severity'] = lr.fit_transform(X['Severity'])
test_df['Severity'] = lr.transform(test_df['Severity'])

In [32]:
for i in range(len(y)):
    if y[i] == 'na':
        y[i] = 0
for i in range(len(y)):
    if y[i] =='0%%':
        y[i] = 0

In [33]:
xgb_n = XGBRegressor().fit(X,y)

[01:02:04] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [37]:
xgb_pred = xgb_n.predict(test_df).round(2)

In [38]:
xgb_pred

array([ 0.05,  0.12,  0.14,  0.04,  0.06,  0.13,  0.17,  0.27,  0.05,
        0.06,  0.06,  0.06,  0.07,  0.4 ,  0.11,  0.23,  0.17,  0.25,
       -0.08,  0.02,  0.25,  0.18,  0.1 ,  0.33,  0.13,  0.07,  0.07,
        0.07,  0.07,  0.05, -0.03,  0.18,  0.18,  0.1 ,  0.18,  0.11,
        0.05,  0.11,  0.47,  0.15,  0.23,  0.14,  0.14,  0.14,  0.03,
        0.09,  0.01,  0.14,  0.24,  0.36,  0.08,  0.23,  0.48],
      dtype=float32)

In [39]:
predictions = pd.Series(xgb_pred)

In [41]:
predictions.to_csv('Attempt1(XGB).csv')

In [42]:
from sklearn.linear_model import LinearRegression

In [44]:
lr = LinearRegression().fit(X,y)
lr_pred = lr.predict(test_df).round(2)
lr_pred

array([ 0.16,  0.28,  0.21,  0.15,  0.07,  0.03,  0.04,  0.14,  0.17,
        0.17,  0.18,  0.06,  0.13,  0.47,  0.16,  0.34,  0.26,  0.5 ,
        0.18, -0.  ,  0.5 ,  0.23,  0.21,  0.28,  0.15,  0.03,  0.03,
        0.03,  0.11,  0.08,  0.25,  0.15,  0.15,  0.13,  0.15,  0.12,
        0.12,  0.12,  0.4 ,  0.16,  0.18,  0.15,  0.15,  0.15,  0.19,
        0.3 ,  0.12,  0.15,  0.27,  0.32,  0.22,  0.26,  0.27])

In [45]:
predictions2 = pd.Series(xgb_pred)
predictions2.to_csv('Attempt2(Linear).csv')